# Problema de los ascensores

#### G05 - Boris Carballa Corredoira, Juan Carlos Villanueva Quirós, Francisco Javier Blázquez Martínez

Problema paralelizado, el edificio consta de varios bloques con varios ascensores.

Frente a las modelizaziones anteriores del problema, que consideran la ejecución secuencial de las acciones del problema en los distintos ascensores, esta versión considera los ascensores como elementos independientes capaces de ejecutar sus acciones simultáneamente. 

Además, esto nos permite cambiar el punto de vista, buscando no ya la solución en la que los ascensores se desplazan menos plantas, sino aquella en la que es mínimo el tiempo de la persona que más tarda en llegar a su destino. Hemos conseguido esto gracias a incorporar el factor tiempo en los estados que analizamos. Lo explicamos en detalle a continuación:

In [17]:
from search import *
import copy
import time

In [18]:
# CONSTANTES
NUM_PLANTAS    = 12 #Para rango por defecto del ascensor
NUM_ASCENSORES = 4
NUM_PERSONAS = 5 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 2
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

In [19]:
%run SolutionManager.ipynb
%run SolutionManagerParalelo.ipynb

Añadimos atributos propios del estado interno de los nodos a explorar como los instantes de llegada a las plantas de las personas (personas_tiempo_llegada) y los ascensores (ascensores_tiempo_llegada) y procuramos su correcta inicialización.

In [20]:
class NodeState:

    def __init__(self, init_personas, ascensor_rango=None):
        self.personas = init_personas
        self.personas_tiempo_llegada = [0 for _ in range(NUM_PERSONAS)]
        self.ascensor_personas = [[] for _ in range(NUM_ASCENSORES)]
        self.ascensores_tiempo_llegada = [0 for _ in range(NUM_ASCENSORES)]
        if ascensor_rango is None:
            self.ascensor_rango = [(0,NUM_PLANTAS) for _ in range(NUM_ASCENSORES)] #[min,max], te deja lo mas arriba/abajo que puede
            self.ascensor_pos = [0 for _ in range(NUM_ASCENSORES)]
        else:
            self.ascensor_rango = ascensor_rango
            self.ascensor_pos = [rango[0] for rango in ascensor_rango]
        
    def __eq__(self, nodo):
        return self.personas == nodo.personas and self.ascensor_pos == nodo.ascensor_pos and self.ascensor_personas == nodo.ascensor_personas and self.ascensores_tiempo_llegada==nodo.ascensores_tiempo_llegada and self.personas_tiempo_llegada==nodo.personas_tiempo_llegada
   
    def __hash__(self):
        return hash((tuple(self.personas), tuple(self.personas_tiempo_llegada), tuple(self.ascensor_pos), tuple(self.ascensores_tiempo_llegada), tuple(tuple(x) for x in self.ascensor_personas)))
    
    def __lt__(self,nodo):
        return True

CAVEAT: ascensor_rango no deberia ser un atributo de NODE_STATE sino de PROBLEM -> GASTAMOS MAS MEMORIA

Las acciones posibles en un estado determinado no requiren modificaciones. Sin embargo, esto hace que podamos ordenar a un ascensor en una planta concreta coger a una persona en esa misma planta aunque su tiempo de llegada sea posterior. Esta acción, a pesar de que supone una espera implícita, la consideramos admisible y contamos en su posterior tratamiento con estos casos que no existían en versiones anteriores del problema.

El resultado de aplicar las acciones permitidas (que son exactamente las mismas que en versiones anteriores) involucra únicamente actualizar como es debido las nuevas variables de marcaje temporal de los distintos elementos. Acciones tales como subir o bajar siempre implican actualizar el tiempo de llegada a la nueva planta, sin embargo, acciones como coger persona (acción que seguimos considerando de tiempo despreciable) pueden también modificar el tiempo de llegada de un ascensor a una planta (o instante hasta el cual el ascensor se encuentra en la planta) siempre que, como hemos mencionado, la acción suponga una espera implícita.

La gran clave de este cambio de visión y objetivo en esta versión del problema es el análisis del coste de una determinada acción. Mientras que en las versiones anteriores el coste era el número de plantas que se hacía subir o bajar al ascensor sobre el que se ejecutaba la acción, ahora, al contar con la ejecución paralela, únicamente tenemos en cuenta al ascensor más lento. El método "path_cost" tiene en cuenta los tiempos de todos los ascensores, no únicamente del que ejecuta la acción. Así, acciones como subir y bajar un determinado ascensor puede tener coste cero siempre y cuando sea plenamente paralelizable.

In [21]:
# Definimos la clase para la representación del problema. Las acciones
# contempladas son hacer al ascensor subir o bajar hasta la planta con
# personas (o planta destino de los ocupantes) más cercana y recoger o
# dejar a una persona en una planta.

class AscensoresSimplificado(Problem) :    
   
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal
        self.analizados = 0

    # Codificación acciones:
    # (BAJAR, i, j) -> El ascensor i baja j plantas
    # (SUBIR, i, j) -> El ascensor i sube j plantas
    # (COGER_PERSONA, i, j) -> El ascensor i coge a la persona j
    # (DEJAR_PERSONA, i, j) -> El ascensor i deja a la persona j
    def actions(self, state):
        
        accs = list()
        personas = state.personas
        
        for num_ascensor in range(0,NUM_ASCENSORES):
            ascensor_pos      = state.ascensor_pos[num_ascensor]
            ascensor_rango    = state.ascensor_rango[num_ascensor]
            ascensor_personas = state.ascensor_personas[num_ascensor]
            ascensor_vacio    = not ascensor_personas
            ascensor_lleno    = (len(ascensor_personas)==CAP_ASCENSOR)

            personas_esta_planta_suben = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]>ascensor_pos]
            personas_esta_planta_bajan = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]<ascensor_pos]


            # 1.- Si alguien ha llegado a su destino o no podemos ir más cerca, forzamos que baje del ascensor
            for persona in ascensor_personas:
                if self.goal[persona]==ascensor_pos or \
                (self.goal[persona]<ascensor_rango[0] and ascensor_pos==ascensor_rango[0]) or \
                (self.goal[persona]>ascensor_rango[1] and ascensor_pos==ascensor_rango[1]):
                    return [(DEJAR_PERSONA,num_ascensor,persona)]


            # 2.- Para la acción subir requerimos que haya que coger o dejar a alguien
            #     más arriba de donde se encuentra el ascensor. Nunca subimos con gente
            #     que desea bajar plantas y además, un ascensor no sube si tiene capacidad
            #     para más personas y en esa planta hay alguien que desea subir (eliminación
            #     de ramas sin perder optimaldad).

            persona_menos_arriba = min([person for person in personas if ascensor_pos<person and person<=ascensor_rango[1] and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo subimos si hay alguna persona por arriba 
            # y no hay gente en esta planta que quiera subir
            if ascensor_vacio:
                if persona_menos_arriba!=NINGUNA and not(personas_esta_planta_suben):
                    accs.append((SUBIR,num_ascensor,persona_menos_arriba-ascensor_pos))

            # Si el ascensor no está vacío comprobamos que sus personas suben y vamos
            # al mínimo entre a donde suben estas y donde hay una persona por encima
            # siempre que no nos quede espacio o no haya gente en esa planta que quiere subir
            elif self.goal[ascensor_personas[0]]>ascensor_pos:
                destino_menos_arriba = min(ascensor_rango[1], min([self.goal[i] for i in ascensor_personas]))

                if ascensor_lleno or not(personas_esta_planta_suben):
                    if persona_menos_arriba==NINGUNA:
                        accs.append((SUBIR,num_ascensor,destino_menos_arriba-ascensor_pos))
                    else:
                        accs.append((SUBIR,num_ascensor,min(persona_menos_arriba, destino_menos_arriba)-ascensor_pos))


            # 3.- La acción bajar es totalmente dual a la acción subir, mismas lógica en
            #     las condiciones. 

            persona_menos_abajo = max([person for person in personas if ascensor_pos>person and person>=ascensor_rango[0] and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo bajamos si hay alguna persona por abajo
            if ascensor_vacio:
                if persona_menos_abajo!=NINGUNA and not(personas_esta_planta_bajan):
                    accs.append((BAJAR,num_ascensor,ascensor_pos-persona_menos_abajo))
            elif self.goal[ascensor_personas[0]]<ascensor_pos:
                destino_menos_abajo = max(ascensor_rango[0], max([self.goal[i] for i in ascensor_personas]))

                if ascensor_lleno or not(personas_esta_planta_bajan):
                    if persona_menos_abajo==NINGUNA:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-destino_menos_abajo))
                    else:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-max(persona_menos_abajo, destino_menos_abajo)))

            # 4.- Acción dejar persona:
            for personaInterior in ascensor_personas:
                accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))

            # 5.- Acción coger persona. Sólo cogemos a personas que suben o bajan, no ambas
            if not ascensor_lleno:
                if ascensor_vacio:
                    if ascensor_pos != ascensor_rango[1]:
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    if ascensor_pos != ascensor_rango[0]:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                else: #si tiene a alguien dentro, ese alguien va hacia adentro del rango, si no forzábamos su salida en 1.-
                    if self.goal[ascensor_personas[0]]<ascensor_pos:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    else:
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))

        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        
        ascensor = action[1]
        
        if action[0]==BAJAR:
            estado_nuevo.ascensor_pos[ascensor] -= action[2]
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        elif action[0]==SUBIR:
            estado_nuevo.ascensor_pos[ascensor] += action[2]
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas[ascensor].append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += max(state.personas_tiempo_llegada[action[2]]-state.ascensores_tiempo_llegada[ascensor],0)
        else:
            estado_nuevo.ascensor_personas[ascensor].remove(action[2])
            estado_nuevo.personas_tiempo_llegada[action[2]] = state.ascensores_tiempo_llegada[ascensor]
       
            if self.goal[action[2]]==state.ascensor_pos[ascensor]:
                estado_nuevo.personas[action[2]] = EN_DESTINO
            else:
                estado_nuevo.personas[action[2]] = state.ascensor_pos[ascensor]
        
        return estado_nuevo
    
    def goal_test(self, state):
        self.analizados +=1
        return state.personas == [EN_DESTINO for _ in state.personas]

    def path_cost(self, c, state1, action, state2):
        return max(state2.ascensores_tiempo_llegada)-max(state1.ascensores_tiempo_llegada)
    
    def h(self,node):
        suma = sum([abs(self.goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        for ascensor in range(0, len(node.state.ascensor_personas)):
            suma += sum([abs(self.goal[persona]-node.state.ascensor_pos[ascensor]) for persona in node.state.ascensor_personas[ascensor]])
        return suma/(CAP_ASCENSOR*NUM_ASCENSORES)
        
    def value(self, state):
        raise NotImplementedError

In [65]:
# Para probar, hay que cambiar los parámetros de arriba también, no solo init
init = NodeState([2,4,1,8,1],[(0,4),(4,8),(4,8),(8,12)])
goal = [3,11,12,1,9]

#init = NodeState([0,4],[(0,4),(4,8),(8,12)])
#goal = [12,8]

problem = AscensoresSimplificado(init, goal)

In [66]:
# Solución con A* 
problem.analizados=0
start = time.time()
answer = astar_search(problem)
end = time.time()
acciones = answer.solution()

In [67]:
acciones_ampliadas = ampliadas_acciones_paralelas(init,acciones)
print("Acciones de los ascensores")
print("Ordenadas por ascensor y momento en el que se realizan")
print()
decodificador_acciones_paralelas(cocina_acciones_paralelas(acciones_ampliadas))

Acciones de los ascensores
Ordenadas por ascensor y momento en el que se realizan

[0,1] -> El ascensor 0 sube 1 planta
[1] -> El ascensor 0 coge a las personas 4, 2
[1,2] -> El ascensor 0 sube 1 planta
[2] -> El ascensor 0 deja a la persona 4
[2] -> El ascensor 0 coge a la persona 0
[2,3] -> El ascensor 0 sube 1 planta
[3] -> El ascensor 0 deja a las personas 0, 2
[3,4] -> El ascensor 0 baja 1 planta
[4] -> El ascensor 0 coge a la persona 4
[4,5] -> El ascensor 0 sube 1 planta
[5] -> El ascensor 0 coge a la persona 2
[5,6] -> El ascensor 0 sube 1 planta
[6] -> El ascensor 0 deja a las personas 4, 2
[6,8] -> El ascensor 0 espera
[8] -> El ascensor 0 coge a la persona 3
[8,11] -> El ascensor 0 baja 3 plantas
[11] -> El ascensor 0 deja a la persona 3
[0] -> El ascensor 1 coge a la persona 1
[0,4] -> El ascensor 1 sube 4 plantas
[4] -> El ascensor 1 deja a la persona 1
[4] -> El ascensor 1 coge a la persona 3
[4,8] -> El ascensor 1 baja 4 plantas
[8] -> El ascensor 1 deja a la persona 3
[

In [68]:
print("Nodos analizados: "+str(problem.analizados))
print("Coste secuencial: "+str(coste_acciones(acciones)))
print("Coste paralelo: "+str(coste_acciones_paralelas(init,acciones)))
print("Tiempo de búsqueda: "+str(end-start)+" s")

Nodos analizados: 1309
Coste secuencial: 35
Coste paralelo: 14
Tiempo de búsqueda: 3.0418548583984375 s


El coste para este mismo problema (tanto en nodos analizados como en tiempo) es menor que el obtenido en la versión anterior pese a que esta no considera la paralelización de los ascensores. Esto es así gracias a que con nuestra nueva función de coste favorecemos siempre las acciones en las que los ascensores están todos siendo utilizados. 

El cambio del paradigma, la pérdida de secuencialidad, hace que se analicen antes aquellas acciones que involucran varios ascensores simultaneamente. Una vez un ascensor esté funcionando, todas las acciones de los restantes ascensores que terminen de ejecutarse antes de que termine la acción de este ascensor tendrán coste cero, lo que gracias al algoritmo A\* implica que serán analizadas con preferencia frente a volver a hacer funcionar ese ascensor.

Distinguimos también entre coste secuencial (el número de plantas que recorren los ascensores del problema) y coste paralelo (número de plantas máximo que tiene que recorrer un ascensor). Gracias a que nuestra heurística es admisible y consistente obtenemos la solución óptima (en el sentido de más rápida de ejecución) al problema planteado inicialmente.

In [71]:
    def h(node):
        suma = sum([abs(goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        for ascensor in range(0, len(node.state.ascensor_personas)):
            suma += sum([abs(goal[persona]-node.state.ascensor_pos[ascensor]) for persona in node.state.ascensor_personas[ascensor]])
        return suma

In [72]:
# CONSTANTES
NUM_PLANTAS    = 40 #Para rango por defecto del ascensor
NUM_ASCENSORES = 4
NUM_PERSONAS = 8 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 2
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

In [73]:
# 8 personas 40 plantas
init = NodeState([8, 32, 15, 24, 17, 3, 18, 21],[(0,10),(10,30),(30,40),(0,10)])
goal = [32, 26, 21, 25, 15, 13, 17, 37]

problem = AscensoresSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem,h).solution()
end = time.time()

In [74]:
acciones_ampliadas = ampliadas_acciones_paralelas(init,acciones)
print("Acciones de los ascensores")
print("Ordenadas por ascensor y momento en el que se realizan")
print()
decodificador_acciones_paralelas(cocina_acciones_paralelas(acciones_ampliadas))

Acciones de los ascensores
Ordenadas por ascensor y momento en el que se realizan

[0,10] -> El ascensor 1 espera
[10] -> El ascensor 1 coge a las personas 5, 0
[10,13] -> El ascensor 1 sube 3 plantas
[13] -> El ascensor 1 deja a la persona 5
[13,15] -> El ascensor 1 sube 2 plantas
[15] -> El ascensor 1 coge a la persona 2
[15,56] -> El ascensor 1 sube 6 plantas
[21] -> El ascensor 1 deja a la persona 2
[21] -> El ascensor 1 coge a la persona 7
[21,54] -> El ascensor 1 sube 9 plantas
[30] -> El ascensor 1 deja a las personas 0, 7
[30] -> El ascensor 1 coge a la persona 1
[30,34] -> El ascensor 1 baja 4 plantas
[34] -> El ascensor 1 deja a la persona 1
[34,78] -> El ascensor 1 baja 8 plantas
[42] -> El ascensor 1 coge a la persona 6
[42,43] -> El ascensor 1 baja 1 planta
[43] -> El ascensor 1 deja a la persona 6
[43] -> El ascensor 1 coge a la persona 4
[43,45] -> El ascensor 1 baja 2 plantas
[45] -> El ascensor 1 deja a la persona 4
[45,54] -> El ascensor 1 sube 9 plantas
[54] -> El as

In [75]:
print("Nodos analizados: "+str(problem.analizados))
print("Coste secuencial: "+str(coste_acciones(acciones)))
print("Coste paralelo: "+str(coste_acciones_paralelas(init,acciones)))
print("Tiempo de búsqueda: "+str(end-start)+" s")

Nodos analizados: 84
Coste secuencial: 66
Coste paralelo: 55
Tiempo de búsqueda: 0.03297924995422363 s


In [76]:
# CONSTANTES
NUM_PLANTAS    = 100 #Para rango por defecto del ascensor
NUM_ASCENSORES = 10
NUM_PERSONAS = 40 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 4
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

In [77]:
# 40 personas 100 plantas
init = NodeState([47, 66, 68, 77, 15, 60, 36, 48,  9, 35, 58, 63, 84, 19, 42, 27, 20,  8, 37, 52, 29, 62, 59, 69, 14, 25, 21, 76, 82, 79, 17, 80, 95, 90, 23, 28, 39,  2, 22, 87],[(0,20),(0,20),(0,20),(20,40),(20,40),(40,60),(40,60),(60,90),(60,90),(90,100)])
goal =           [59,  1, 43, 28, 90, 34, 20, 92, 85,  5, 13, 39, 46,  4, 38, 86, 35, 65, 48, 27, 89, 50, 37, 42, 32, 36, 68, 74, 72, 26, 82,  6, 97, 79, 99, 56, 91, 44, 66, 88]

problem = AscensoresSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem,h).solution()
end = time.time()

In [78]:
acciones_ampliadas = ampliadas_acciones_paralelas(init,acciones)
print("Acciones de los ascensores")
print("Ordenadas por ascensor y momento en el que se realizan")
print()
decodificador_acciones_paralelas(cocina_acciones_paralelas(acciones_ampliadas))

Acciones de los ascensores
Ordenadas por ascensor y momento en el que se realizan

[0,2] -> El ascensor 2 sube 2 plantas
[2] -> El ascensor 2 coge a la persona 37
[2,8] -> El ascensor 2 sube 6 plantas
[8] -> El ascensor 2 coge a la persona 17
[8,9] -> El ascensor 2 sube 1 planta
[9] -> El ascensor 2 coge a la persona 8
[9,14] -> El ascensor 2 sube 5 plantas
[14] -> El ascensor 2 coge a la persona 24
[14,71] -> El ascensor 2 sube 6 plantas
[20] -> El ascensor 2 deja a las personas 37, 17, 8, 24
[20,222] -> El ascensor 2 espera
[222] -> El ascensor 2 coge a las personas 31, 10, 9, 1
[222,904] -> El ascensor 2 baja 7 plantas
[229] -> El ascensor 2 deja a la persona 10
[229,236] -> El ascensor 2 baja 7 plantas
[236] -> El ascensor 2 deja a la persona 31
[236,237] -> El ascensor 2 baja 1 planta
[237] -> El ascensor 2 deja a la persona 9
[237,241] -> El ascensor 2 baja 4 plantas
[241] -> El ascensor 2 deja a la persona 1
[241,255] -> El ascensor 2 sube 14 plantas
[255] -> El ascensor 2 coge 

In [79]:
print("Nodos analizados: "+str(problem.analizados))
print("Coste secuencial: "+str(coste_acciones(acciones)))
print("Coste paralelo: "+str(coste_acciones_paralelas(init,acciones)))
print("Tiempo de búsqueda: "+str(end-start)+" s")

Nodos analizados: 389
Coste secuencial: 1153
Coste paralelo: 332
Tiempo de búsqueda: 2.821626663208008 s


In [80]:
# CONSTANTES
NUM_PLANTAS    = 12 #Para rango por defecto del ascensor
NUM_ASCENSORES = 4
NUM_PERSONAS = 10 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 2
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

In [42]:
init = NodeState([3, 1, 5, 10, 8, 11, 6, 12, 9, 4],[(0,4),(4,8),(4,8),(8,12)])
goal = [2, 3, 6, 1, 11, 9, 8, 10, 4, 12]

problem = AscensoresSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem,h).solution()
end = time.time()

In [43]:
acciones_ampliadas = ampliadas_acciones_paralelas(init,acciones)
print("Acciones de los ascensores")
print("Ordenadas por ascensor y momento en el que se realizan")
print()
decodificador_acciones_paralelas(cocina_acciones_paralelas(acciones_ampliadas))

Acciones de los ascensores
Ordenadas por ascensor y momento en el que se realizan

[0,1] -> El ascensor 0 sube 1 planta
[1] -> El ascensor 0 coge a la persona 1
[1,3] -> El ascensor 0 sube 2 plantas
[3] -> El ascensor 0 deja a la persona 1
[3] -> El ascensor 0 coge a la persona 0
[3,4] -> El ascensor 0 baja 1 planta
[4] -> El ascensor 0 deja a la persona 0
[4,6] -> El ascensor 0 sube 2 plantas
[6,14] -> El ascensor 0 espera
[14] -> El ascensor 0 coge a la persona 3
[14,17] -> El ascensor 0 baja 3 plantas
[17] -> El ascensor 0 deja a la persona 3
[0] -> El ascensor 2 coge a la persona 9
[0,1] -> El ascensor 2 sube 1 planta
[1] -> El ascensor 2 coge a la persona 2
[1,2] -> El ascensor 2 sube 1 planta
[2] -> El ascensor 2 deja a la persona 2
[2] -> El ascensor 2 coge a la persona 6
[2,4] -> El ascensor 2 sube 2 plantas
[4] -> El ascensor 2 deja a las personas 9, 6
[4,10] -> El ascensor 2 espera
[10] -> El ascensor 2 coge a las personas 8, 3
[10,14] -> El ascensor 2 baja 4 plantas
[14] -> 

In [44]:
print("Nodos analizados: "+str(problem.analizados))
print("Coste secuencial: "+str(coste_acciones(acciones)))
print("Coste paralelo: "+str(coste_acciones_paralelas(init,acciones)))
print("Tiempo de búsqueda: "+str(end-start)+" s")

Nodos analizados: 65
Coste secuencial: 31
Coste paralelo: 17
Tiempo de búsqueda: 0.034656524658203125 s
